In [1]:
import torch
from torchvision import models

from matplotlib import pyplot as plt 

import numpy as np 

# Introduction


In the following, we will develop an approach to compute the size of the receptive field of any output neuron from a PyTorch model.

We divide the implementation of the approach into several functions, and your task is to implement these functions. Once these steps are implemented, you will use the implementation to calculate the size of the receptive fields in various configurations.



**This programming exercise consists of 5 tasks (4 programming and 1 questionaire), and the exercise is worth 45  points in total; the distribution of the points is 10 + 10 + 10 + 10 + 5 Points;** These tasks are labelled as Task 1 to Task 5.
<!-- 


- Assumption 1: gradient is not varnish
- Assumption 2: input has 2 spatial dimensions: width and height.  We assume that width = height


4 Tasks with 30 = 5 + 10 + 10 + 5 P -->

# Core Implementation of the Approach

In [2]:
# We set the seed to make the dummy input below comparable across runs.
torch.manual_seed(1)

# This is dummy input that we will use throughout the sheet.
# Because the approach does NOT need to use actual input, we therefore use random input.
# In fact, we use 10 random inputs and aggregate the quantity of interest from these 10 inputs.
# This aggregation makes sure that we have a good estimate of the quanity.
x = torch.randn((10, 3, 224, 224))

In [3]:
def calculate_receptive_field_size(forward_func, x):
    """
    This function is the main function that integrates three necessary functions together
    to calcuate the size of the receptive field.
    
    Your task is to implement these functions (defined outside this main function).
    To aid the implemenation, we provide a number of assertions in this main function
    to aid/verify your solutions.
    """

    # Step 1
    dependency = find_dependency_between_output_and_input(forward_func, x)
    assert dependency is not None

    # sum over channel and batch dimension
    dependency = dependency.sum(axis=1).sum(axis=0)

    # Step 2
    mask = criteria(dependency)
    assert mask.shape == dependency.shape 

    # Step 3
    size =  calculate_size(mask)
    assert (size * 10) % 10 == 0, "We should have a whole number."

    return int(size)

In [4]:
def _select_output_neuron(z):
    batch_size, dimensions, h, w =  z.shape

    assert h == w, "Output's spatial size is square."

    # Because we are interested in the size of the receptive at that layer,
    # we therefore select the neuron in the middle of the feature map.
    # This selection also provents the influence of padding on the calculation.
    zj = z[:, :, h//2, h//2]

    return zj

def find_dependency_between_output_and_input(forward_func: torch.nn.Sequential, x: torch.Tensor) -> torch.Tensor:
    """
    Task 1 (10P):
    
    Given input `x`, compute the quantity of interest from an output neuron from `forward_func`.

    Hint: You can use `_select_output_neuron` to select such an output neuron.
    """
    x = x.clone()
    x.requires_grad = True
    forward = forward_func(x)
    selected = _select_output_neuron(forward)
    selected = selected.mean()
    selected.backward()
    return x.grad
#     forward = forward_func(x)
#     selected = _select_output_neuron(forward)
    
#     layer_list = []
#     for name, module in forward_func.named_modules():
#         if isinstance(module, torch.nn.Conv2d):
#             # print(name, module.kernel_size, module.padding)
#             layer_list.append(torch.nn.ConvTranspose2d(module.out_channels, module.in_channels, kernel_size=module.kernel_size, padding=module.padding, stride=module.stride))
#         elif(isinstance(module, torch.nn.MaxPool2d)):
#             print(name, module.kernel_size, module.padding)
#             # layer_list.append(torch.nn.MaxUnpool2d(kernel_size=module.kernel_size, padding=module.padding))
#             layer_list.append(torch.nn.Upsample(scale_factor=module.kernel_size**2, mode='bilinear'))
#         elif(isinstance(module, torch.nn.Identity)):
#             layer_list.append(torch.nn.Identity())
#     layers = torch.nn.Sequential(*layer_list[::-1])
#     res = torch.reshape(selected, (selected.shape[0], selected.shape[1], 1, 1))
#    return layers(res)

In [5]:
def criteria(inp: torch.Tensor) -> torch.Tensor:
    """
    Task 2 (10P):

    The function implements a thresholding mechanism that discards spatial entries that the quantity of interest is zero.

    Hint: the function should return a tensor whose entires are either zero or one.
    """

    # YOUR CODE HERE
    res = torch.where(inp != 0, 1, 0)
    return res

In [6]:
def calculate_size(binary_mask: torch.Tensor) -> int:
    """
    Task 3 (10P):

    The function finds the size (e.g., height or width, NOT the area) of the active region in the given binary mask).

    Hint: Recall that the shape of the input is square; height and width have the same value.
        Leverage this fact and find the size.
    """
    
    assert (binary_mask == 1).sum() + (binary_mask == 0).sum() \
        == binary_mask.shape[0] * binary_mask.shape[0], "We should have a binary mask."

    # YOUR CODE HERE
#     indices = binary_mask.nonzero()
#     size = indices.size(dim=0) ** 0.5

    return binary_mask.sum() ** (1/len(binary_mask.shape))

# Verifying the Implementation on Simple Cases

We first verify our implemenation on simple cases. These cases have two convolution layers with the following configuations
## **Case 1**: $(k_1=3, s_1=1)\rightarrow (k_2=1, s_2=1)$

In [7]:
simple_model_case_1 = torch.nn.Sequential(
    torch.nn.Conv2d(3, 1, kernel_size=3, stride=1),
    torch.nn.Conv2d(1, 1, kernel_size=1, stride=2),
)

In [8]:
# If the implementation is correct, we should expect the function outputs `3`.
calculate_receptive_field_size(
    simple_model_case_1, x
)

3

## Case 2: $(k_1=3, s_1=1)\rightarrow (k_2=3, s_2=1)$

In [9]:
simple_model_case_2 = torch.nn.Sequential(
    torch.nn.Conv2d(3, 1, kernel_size=3, stride=1),
    torch.nn.Conv2d(1, 1, kernel_size=3, stride=2),
)

In [10]:
# If the implementation is correct, we should expect the function outputs `5`.
calculate_receptive_field_size(
    simple_model_case_2, x
)

5

# Calculating Receptive Field Size of Neurons from VGG16 

In [11]:
# We first load a pretrained VGG16 model from the TorchVision model respository.
vgg16 = models.vgg16(pretrained=True)

# We also set the model to the evaluation mode to prevent
# some statistics of the model get updated during forward passes.
vgg16 = vgg16.eval()

In [12]:
# Explore the structure of VGG16
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

The output above shows that `VGG16` (from PytorchVision) are composed of three modules, namely
- `features` mainly contains a sequence of convolution and poolying layers; one can interpret the module as a feature extractor.
- `avgpool` is a average pooling over spatial dimensions.
- `classifier` is a sequence of linear layers that is responsible to making final predictions.

## Preventing the Quantity of Interest Varnish

Under some circumstances, the quantity of our interest could be zero for activation functions like ReLU. From architectures (e.g. `VGG16`) that rely on such activation functions, we therefore have to prevent circumstances to happen.

Because we are only interested in computing the quanity (and NOT the output of the model), we can mitigate the effect by replacing ReLU in `VGG16` with the identity function.

In [13]:
def replace_activation_in_module_to_identity(module: torch.nn.Sequential):
    """
    The function replaces all ReLU activation to be identity function.
    It prevents the quanity of interest varnish.
    
    Remark:
    In practice, we have to be VERY careful on this type of reassignment.
    This is because now we have a different model, and it certianly produces different output from the original one.
    For our purpose, this replacement is ok because we do NOT rely directly on the exact output value. 
    """

    
    for i in range(len(module)):
        layer = module[i]
    
        if isinstance(layer, torch.nn.ReLU):

            module[i] = torch.nn.Identity()

In [14]:
replace_activation_in_module_to_identity(vgg16.features)

In [15]:
replace_activation_in_module_to_identity(vgg16.classifier)

In [16]:
# Verify that ReLU are replaced with Identity
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Identity()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): Identity()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): Identity()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): Identity()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): Identity()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): Identity()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): Identity()
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d(256, 512, kernel_siz

From the output above, we can see that now all ReLU's are replaced by `Identity()`.

## Helper Function to Select Parts of Model

In [17]:
def slice_module(module: torch.nn.Sequential, layer_ix: int) -> torch.nn.Sequential:

    """
    Task 4 (10P): 
    
    The function selects part of the module (torch.nn.Sequential object)
    from the first layer (layer_ix=0) to the `layer_ix`-th (inclusive).

    Hints:
    1. Python starts index at 0;
    2. Recall that Python's List Slicing does NOT include the last entry.
       For example, [0, 1, 2][:2] is [0, 1].
    """

    import torch.nn as nn

    layers = vgg16.features
    classifier = vgg16.classifier
    
    if layer_ix <= 30:
        return layers[:layer_ix+1]
    elif layer_ix==31:
        return layers[:31].add_module(vgg16.avgpool)
    else:
        before_avg = layers[:31].add_module(vgg16.avgpool)
        return before_avg.add_module(classifier[:layer_ix-31])

In [18]:
# Verify that `slice_module` returns an object with correct type.
assert isinstance(
    slice_module(vgg16.features, 4),
    torch.nn.Sequential
), "The function should returns a `torch.nn.Sequential` object."

In [19]:
# Verity that if we slide vgg16.features at layer_ix=1,
# we should expect only a convolution layer and its proceeding activation function Identity().
slice_module(vgg16.features, 1)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Identity()
)

In [20]:
# Verity that if we slide vgg16.features at layer_ix=4,
# we should expect 2x[Conv2d, Identity] and MaxPool2d.
slice_module(vgg16.features, 4)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Identity()
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Identity()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

## Calculating Receptive Field at Various Layer of VGG16

In [21]:
# Suppose we are interested in these layers.
# We assume that these layers are part of vgg16.features.
# If not, one has to change the module given to slice_module(...) in the cell below accordingly.
LAYERS_OF_INTEREST = [2, 4, 14, 16, 21, 28, 30]

In [22]:
for layer_ix in LAYERS_OF_INTEREST:
    # We slice vgg16.features.
    submodule_until_layer_ix = slice_module(vgg16.features, layer_ix)
    
    # Use the function that we implement to calculate the size of the receptive field.
    rf = calculate_receptive_field_size(submodule_until_layer_ix, x)
    
    
    print(f"Layer {layer_ix:2d}: RF={rf:03.0f} [last-layer={submodule_until_layer_ix[-1]}]")

Layer  2: RF=005 [last-layer=Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]


c:\users\87290\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Layer  4: RF=006 [last-layer=MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)]
Layer 14: RF=040 [last-layer=Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
Layer 16: RF=044 [last-layer=MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)]
Layer 21: RF=092 [last-layer=Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
Layer 28: RF=196 [last-layer=Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
Layer 30: RF=212 [last-layer=MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)]


From the output, we see that the receptive field gets larger when we move closer to output layer. This means that neurons in later layers (high-level) see larger parts of the input than neurons in early layers (low-level).

Said differently, neurons in early layers can only detect simple or low-level)  features (due to the restrict size of their receptive field. Conversely, neurons in later layers can becomre morespecialize in detecting complex patterns or distinct objects (high-level features).

# Limitation of the Approach

Our current approach on calculating the size of receptive fields requires that the quanity of interest is computable and not varnish.

But, in practice, whether the quantity of interest varnishes is sometimes difficult ot realize. This issue leads to an important limitation that one should be aware of.


We demonstrate the limitation through a set of examples. In these examples, we look at the receptive field of `VGG16.features`'s Layer 30. From the output, we see that the size of its receptive field is `212`.

Instead of using the input with spatial dimensions `224x224`, we vary these dimensions and observe the size of the receptive fields in these configuration.

In [23]:
submodule_until_layer_30 = slice_module(vgg16.features, 30)

for size in [56, 98, 112, 224, 448]:
    # Create input with different spatial size
    xsmall = torch.randn(10, 3, size, size)

    rf = calculate_receptive_field_size(submodule_until_layer_30, xsmall)
    
    print(f"input size=({size:03d},{size:03d}): RF(Layer 30)={rf:3.0f}")

input size=(056,056): RF(Layer 30)= 56
input size=(098,098): RF(Layer 30)= 98
input size=(112,112): RF(Layer 30)=112
input size=(224,224): RF(Layer 30)=212
input size=(448,448): RF(Layer 30)=212


**Task 5 (5 P):** Are the receptive field sizes of Layer 30 the same when using `x` and smaller versions of `x`?

If NOT, why and how are they different?

*Answer*: ....